In [2]:
import pandas as pd
import numpy as np
from scipy.signal import medfilt

# Carica
ACC_CSV = r"C:\Users\eleut\OneDrive\Desktop\Cow during rumination\video2\segment_01.csv"
acc = pd.read_csv(ACC_CSV)
acc['time_s'] = acc['time_ms'].astype(float) / 1000.0

# Sampling rate e finestra 1 s
fs = int(round(1 / acc['time_s'].diff().median()))

# Rolling‑std
acc['std_nm']  = acc['norm_mavg'].rolling(fs, center=True).std()
acc['std_xsg'] = acc['x_sg'].rolling(fs, center=True).std()

# Soglie 10° percentile
thr_nm  = np.percentile(acc['std_nm'].dropna(), 10)
thr_xsg = np.percentile(acc['std_xsg'].dropna(), 10)

# Flag pausa = OR dei due criteri
acc['pause_flag'] = (acc['std_nm']  < thr_nm) | (acc['std_xsg'] < thr_xsg)

# Smussa con filtro mediano
acc['pause_flag_sm'] = medfilt(acc['pause_flag'].astype(int), kernel_size=5).astype(bool)

# Segmentazione e filtro durata ≥2s
acc['seg'] = (acc['pause_flag_sm'] != acc['pause_flag_sm'].shift(1)).cumsum()
segs = (acc[acc['pause_flag_sm']]
        .groupby('seg')
        .agg(start_s=('time_s','min'), end_s=('time_s','max'))
        .assign(dur=lambda df: df.end_s - df.start_s)
        .query('dur >= 2.0')
        .reset_index(drop=True))

# Stampa
print("=== Pause rilevate (OR + 10° pct, dur>=2s) ===")
for i, row in segs.iterrows():
    print(f"#{i+1}: {row.start_s:.2f}s → {row.end_s:.2f}s  (dur {row.dur:.2f}s)")

=== Pause rilevate (OR + 10° pct, dur>=2s) ===
#1: 20.36s → 23.35s  (dur 2.99s)
#2: 69.76s → 74.97s  (dur 5.21s)
#3: 111.87s → 116.09s  (dur 4.22s)
#4: 166.33s → 170.43s  (dur 4.09s)
